In [ ]:
import numpy as np
import plotly.io as pio

import Config.constants as cnst
import Config.experiment_config as cnfg
from Analysis.detector_comparison.DetectorComparisonAnalyzer import DetectorComparisonAnalyzer
from Visualization.distributions_grid import distributions_grid
from Visualization.p_value_heatmap import heatmap_grid

pio.renderers.default = "notebook"

In [ ]:
DATASET = "Lund2013"
COL_MAPPER = lambda col: col[:col.index("ector")] if "ector" in col else col
VERBOSE = True

STAT_TEST_NAME = "Mann-Whitney"
CRITICAL_VALUE = 0.05
CORRECTION = "bonferroni"

SHOW_DISTRIBUTIONS = True
SHOW_P_VALUES = True

In [ ]:
%%capture --no-stdout

_, events, _, event_matches, comparison_columns = DetectorComparisonAnalyzer.preprocess_dataset(DATASET,
                                                                         column_mapper=COL_MAPPER,
                                                                         verbose=VERBOSE)

# events = events.map(lambda cell: [e for e in cell if e.event_label == cnfg.EVENT_LABELS.FIXATION])
# event_matches = event_matches.map(
#     lambda cell: {k: v for k, v in cell.items() if k.event_label == cnfg.EVENT_LABELS.FIXATION}
#     if cell is not None else None)

comparison_columns

In [ ]:
fixation_metrics = DetectorComparisonAnalyzer.analyze(events, event_matches,
                                    ignore_events={v for v in cnfg.EVENT_LABELS if v != cnfg.EVENT_LABELS.FIXATION},
                                    verbose=VERBOSE)
event_features = fixation_metrics["Event Features"]
event_matching_ratios = fixation_metrics["Event Matching Ratios"]
event_matching_feature_diffs = fixation_metrics["Event Matching Feature Diffs"]

# Fixations' Feature Distributions

In [ ]:
print(f"Available fixation features: {list(event_features.keys())}")

In [ ]:
if SHOW_DISTRIBUTIONS:
    for feature in event_features.keys():
        if feature not in {"Duration"}:
            continue
        title = f"{DATASET.upper()}:\t\tFixations' {feature.title()} Distribution"
        distributions_grid(
            data=event_features[feature],
            title=title,
            show_counts=feature == "Counts",
        ).show()

In [ ]:
if SHOW_P_VALUES:
    for feature in event_features.keys():
        if feature == "Counts":
            continue
        feature_values = event_features[feature].map(lambda cell: [v for v in cell if not np.isnan(v)])
        stat_test_res = DetectorComparisonAnalyzer.event_feature_statistical_comparison(feature_values, STAT_TEST_NAME)
        p_values = stat_test_res.xs(cnst.P_VALUE, axis=1, level=2)
        heatmap_grid(
            p_values,
            critical_value=CRITICAL_VALUE,
            correction=CORRECTION,
            add_annotations=True,
            ignore_above_critical=True
        ).show()

# Fixation Matching Ratio

In [ ]:
if SHOW_DISTRIBUTIONS:
    distributions_grid(
        data=event_matching_ratios["Match Ratio"][comparison_columns],
        title=f"{DATASET.upper()}:\t\tFixation-Matching Ratios",
        pdf_min_val=0,
        pdf_max_val=100,
        column_title_mapper=lambda col: f"{col[0]}→{col[1]}"
    ).show()

# Matched-Fixations' Feature Differences

In [0]:
print(f"Available matched-fixation feature differences: {list(event_matching_feature_diffs.keys())}")

In [ ]:
if SHOW_DISTRIBUTIONS:
    for feature in event_matching_feature_diffs.keys():
        if feature not in ['Duration', 'Onset Jitter', 'Overlap Time', 'L2 Timing Jitter', 'Offset Jitter', 'IoU']:
            continue
        distributions_grid(
            data=event_matching_feature_diffs[feature][comparison_columns],
            title=f"{DATASET.upper()}:\t\tMatched-Fixations' {feature.title()} Distribution",
            column_title_mapper=lambda col: f"{col[0]}→{col[1]}",
            pdf_min_val=0 if feature == "IoU" else None,
            pdf_max_val=1 if feature == "IoU" else None,
        ).show()